In [1]:
import os 
os.environ['GEOMSTATS_BACKEND'] = 'pytorch'

import geomstats._backend as gs 
import gpytorch

import geometric_kernels.torch 
from mdgp.experiments.experiment_utils.reproducibility import ExperimentConfig

INFO: Using pytorch backend
/home/kacperwyrwal/miniconda3/envs/mdgp_requirements_test7/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO: Created a temporary directory at /tmp/tmpavow73fm
INFO: Writing /tmp/tmpavow73fm/_remote_module_non_scriptable.py


In [2]:
config = ExperimentConfig()

In [3]:
config.to_json("../../experiment_tree_configs/benchmarking/")

In [4]:
config_name = "../../experiment_tree_configs/benchmarking/geometric_head_single.json"
# config_name = "../../experiment_tree_configs/benchmarking/euclidean_single.json"
# config_name = "../../experiment_tree_configs/benchmarking/residual_geometric_single.json"

experiment_config = ExperimentConfig.from_json(config_name)
# experiment_config = ExperimentConfig.from_json("../../experiment_tree_configs/benchmarking/residual_geometric_single.json")

In [5]:
import os 
from torch import set_default_dtype, float64
from torch.optim import Adam  
from gpytorch.mlls import DeepApproximateMLL, VariationalELBO, ExactMarginalLogLikelihood
from mdgp.experiments.experiment_utils.data import get_data 
from mdgp.experiments.experiment_utils.model import create_model
from mdgp.experiments.experiment_utils.logging import CSVLogger, finalize 
from mdgp.experiments.experiment_utils.training import fit, test_step
from mdgp.experiments.experiment_utils import set_experiment_seed
set_default_dtype(float64)

In [6]:
def run_experiment(experiment_config, dir_path):
    print(f"Running experiment with the config: {os.path.join(dir_path, experiment_config.file_name)}")
    # 0. Unpack arguments
    model_args, data_args, training_args = experiment_config.model_arguments, experiment_config.data_arguments, experiment_config.training_arguments

    # 1. Get data 
    train_inputs, train_targets, val_inputs, val_targets, test_inputs, test_targets = get_data(data_args=data_args)

    # 2. Create model, criterion, and optimizer 
    model = create_model(model_args=model_args, train_x=train_inputs, train_y=train_targets)
    print(model)
    if model_args.model_name == 'exact':
        mll = ExactMarginalLogLikelihood(likelihood=model.likelihood, model=model)
    else:
        mll = DeepApproximateMLL(
            VariationalELBO(likelihood=model.likelihood, model=model, num_data=data_args.num_train)
        )
    optimizer = Adam(model.parameters(), maximize=True, lr=0.01) # Maximize because we are working with ELBO not negative ELBO 
    
    # 4. Train and validate model
    print("Training...")
    train_csv_logger = CSVLogger(root_dir=os.path.join(dir_path, 'train')) 
    val_csv_logger = CSVLogger(root_dir=os.path.join(dir_path, 'val')) 
    train_loggers = [train_csv_logger]
    val_loggers = [val_csv_logger]
    model = fit(model=model, optimizer=optimizer, criterion=mll, train_loggers=train_loggers, 
                val_loggers=val_loggers, train_inputs=train_inputs, train_targets=train_targets,
                val_inputs=val_inputs, val_targets=val_targets, training_args=training_args)

    # make sure logger files are saved
    finalize(loggers=[*val_loggers, *train_loggers])

    # # 5. Test model 
    print("Testing...")
    test_csv_logger = CSVLogger(root_dir=os.path.join(dir_path, 'test'))
    test_loggers = [test_csv_logger]
    test_metrics = test_step(model=model, inputs=test_inputs, targets=test_targets, sample_hidden=training_args.sample_hidden, 
                             loggers=test_loggers, train_targets=train_targets)
    # make sure logger files are saved
    finalize(loggers=test_loggers)
    print(test_metrics)
    print("Done!")


In [7]:
set_experiment_seed(1)

run_experiment(experiment_config=experiment_config, dir_path='./')

INFO: Global seed set to 1
INFO: Global seed set to 1
/home/kacperwyrwal/miniconda3/envs/mdgp_requirements_test7/lib/python3.11/site-packages/mdgp/samplers/rff_sampler.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self._num_features = self.compute_features(torch.tensor(self.base_kernel.space.random_point()[None])).shape[-1]


Running experiment with the config: ./geometric_head_single.json
GeometricHeadDeepGP(
  (hidden_layers): ModuleList(
    (0): GeometricDeepGPLayer(
      (variational_strategy): VariationalStrategy(
        (_variational_distribution): CholeskyVariationalDistribution()
      )
      (mean_module): LinearMean()
      (covar_module): ScaleKernel(
        (base_kernel): _GeometricMaternKernel(
          (raw_lengthscale_constraint): Positive()
          (raw_nu_constraint): Positive()
        )
        (outputscale_prior): GammaPrior()
        (raw_outputscale_constraint): Positive()
      )
      (sampler): PosteriorSampler(
        (rff_sampler): RFFSampler(
          (covar_module): ScaleKernel(
            (base_kernel): _GeometricMaternKernel(
              (raw_lengthscale_constraint): Positive()
              (raw_nu_constraint): Positive()
            )
            (outputscale_prior): GammaPrior()
            (raw_outputscale_constraint): Positive()
          )
          (base_ke

Fitting:   0%|          | 0/1000 [00:00<?, ?it/s]/home/kacperwyrwal/miniconda3/envs/mdgp_requirements_test7/lib/python3.11/site-packages/lightning/fabric/loggers/csv_logs.py:195: UserWarning: Experiment logs directory ./train exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
  rank_zero_warn(
/home/kacperwyrwal/miniconda3/envs/mdgp_requirements_test7/lib/python3.11/site-packages/lightning/fabric/loggers/csv_logs.py:195: UserWarning: Experiment logs directory ./val exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
  rank_zero_warn(
Fitting: 100%|██████████| 1000/1000 [02:03<00:00,  8.11it/s, elbo=1.35, nlpd=-1.41, smse=0.0773]


Testing...
{'expected_log_probability': tensor(1.4061), 'mean_absolute_error': tensor(0.0412), 'mean_squared_error': tensor(0.0033), 'standardized_mean_squared_error': tensor(0.0733), 'mean_standardized_log_loss': tensor(-1.3016), 'quantile_coverage_error': tensor(0.0021), 'negative_log_predictive_density': tensor(-1.4417)}
Done!


/home/kacperwyrwal/miniconda3/envs/mdgp_requirements_test7/lib/python3.11/site-packages/lightning/fabric/loggers/csv_logs.py:195: UserWarning: Experiment logs directory ./test exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
  rank_zero_warn(
